# Imports

In [187]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import zipfile
import getpass
from time import sleep
from random import randint
import spotipy

# Spotify API

## User ID and Secret

In [281]:
CLIENT_ID = getpass.getpass()
CLIENT_SECRET = getpass.getpass()

 ································
 ································


## Token

In [282]:
token = spotipy.oauth2.SpotifyClientCredentials(client_id = CLIENT_ID, client_secret = CLIENT_SECRET)

In [283]:
cache_token = token.get_access_token()

C:\Users\alfon\AppData\Local\Temp\ipykernel_1512\4134625355.py:1: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  cache_token = token.get_access_token()


In [ ]:
cache_token

{'access_token': 'BQDwlR3SR4Ih2Dg02I18eBWOM3bwgNuXLzM-AbXA3ZXJPE3YInw-IvDMSpEAIgB6DGtdiGyBBGSnhZV4eHzTyC1146f9uN28jpLu6FIE-CDacdwXyKc',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'expires_at': 1661188558}

In [285]:
sp = spotipy.Spotify(cache_token["access_token"])

## Request

In [ ]:
sp.user_playlist_tracks("acclaimed music", "1G8IpkZKobrIlXcVPoSIuf")

{'href': 'https://api.spotify.com/v1/playlists/1G8IpkZKobrIlXcVPoSIuf/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2020-11-29T15:02:07Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/acclaimedmusic'},
    'href': 'https://api.spotify.com/v1/users/acclaimedmusic',
    'id': 'acclaimedmusic',
    'type': 'user',
    'uri': 'spotify:user:acclaimedmusic'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/74ASZWbe4lXaubB36ztrGX'},
       'href': 'https://api.spotify.com/v1/artists/74ASZWbe4lXaubB36ztrGX',
       'id': '74ASZWbe4lXaubB36ztrGX',
       'name': 'Bob Dylan',
       'type': 'artist',
       'uri': 'spotify:artist:74ASZWbe4lXaubB36ztrGX'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
     

### Function to get the data from the first 100 songs

In [306]:
def analyze_playlist(creator, playlist_id):
    playlist_features_list = ["Songs", "Artists"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)

    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        playlist_features = {}
        playlist_features["Artists"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["Songs"] = track["track"]["name"]
        
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)

    return playlist_df

In [312]:
not_hot = analyze_playlist("acclaimed music", "1G8IpkZKobrIlXcVPoSIuf")
not_hot

,Songs,Artists
0,Like a Rolling Stone,Bob Dylan
1,Smells Like Teen Spirit,Nirvana
2,A Day In The Life - Remastered,The Beatles
3,Good Vibrations (Mono),The Beach Boys
4,Johnny B Goode,Chuck Berry
...,...,...
95,Life On Mars? - 1999 Remastered Version,David Bowie
96,Reach Out I'll Be There - Single Version / Mono,Four Tops
97,That's All Right Mama,Elvis Presley
98,Get Up (I Feel Like Being A) Sex Machine - Pt....,James Brown


### Function to get the other 4900 songs

In [310]:
results = sp.playlist_items("1G8IpkZKobrIlXcVPoSIuf")
next_pages = 49
artists = []
songs = []

for i in range(next_pages):
    tracks = sp.next(results)
    for y in range(0,100):
        try:
            track = tracks['items'][y]['track']['name']
            songs.append(track)
            artist = tracks['items'][y]['track']['artists'][0]['name']
            artists.append(artist)
        except:
            continue

df = pd.DataFrame({"Songs":songs, "Artists":artists})          
df

,Songs,Artists
0,Heroin,The Velvet Underground
1,Whole Lotta Shakin' Goin' On,Jerry Lee Lewis
2,Thunder Road,Bruce Springsteen
3,One,U2
4,I'm Waiting For The Man - Album Version (Stereo),The Velvet Underground
...,...,...
4895,Ms. Jackson,Outkast
4896,Walk on By,Dionne Warwick
4897,Folsom Prison Blues,Johnny Cash
4898,Sign 'O' the Times - Single Version,Prince


### Concat and save

In [313]:
not_hot = pd.concat([not_hot, df], ignore_index = True)
not_hot

,Songs,Artists
0,Like a Rolling Stone,Bob Dylan
1,Smells Like Teen Spirit,Nirvana
2,A Day In The Life - Remastered,The Beatles
3,Good Vibrations (Mono),The Beach Boys
4,Johnny B Goode,Chuck Berry
...,...,...
4995,Ms. Jackson,Outkast
4996,Walk on By,Dionne Warwick
4997,Folsom Prison Blues,Johnny Cash
4998,Sign 'O' the Times - Single Version,Prince


In [314]:
not_hot.to_csv("Not_Hot.csv")